In [1]:
import pulp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# to ignore warnings
import warnings
warnings.filterwarnings('ignore')

# to display all rows columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [2]:
def fantasy_team(df):
    df = df.join(pd.get_dummies(df[['Role', 'Team']]))
    df['Playing_xi'] = 1
    
    # define decision variable
    pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df.index), cat = 'Binary' )

    # define objective function
    mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Points'] for obj in df.index])

    # define constraints
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Credits']] for obj in df.index) <= 100

    # 11 players
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Playing_xi']] for obj in df.index) == 11

    # team constraints
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_IND']] for obj in df.index) >= 4
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_IND']] for obj in df.index) <= 7

    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_SL']] for obj in df.index) >= 4
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_SL']] for obj in df.index) <= 7

    # role constraints
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) >= 1
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) <= 4

    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) >= 3
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) <= 6

    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) >= 3
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) <= 6

    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) >= 1
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) <= 4
    
    mo.solve()
    
    # output as a column in df
    df['pickup_status'] = 0

    for obj in df.index:
        if pickup_status[obj].varValue:
            df['pickup_status'][obj] = 1
            
    fdf = df[df['pickup_status'] == 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False).reset_index(drop = True)
    return fdf

# IND VS SL 1st T20I

In [3]:
df1 = pd.read_csv('ind_sl_t20i_1st_2022_02_24.csv')

In [4]:
df1.head()

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,PlayingXI,Points,DT
0,3692,38632,I Kishan,10276,IND,2,WK,73.23,8.5,0,1,141,1
1,3692,38632,S Iyer,9428,IND,2,BAT,71.23,8.5,0,1,84,1
2,3692,38632,B Kumar,1726,IND,2,BOWL,22.17,8.5,0,1,76,1
3,3692,38632,C Asalanka,10934,SL,5,BAT,64.60,8.5,0,1,70,1
4,3692,38632,R Sharma,576,IND,2,BAT,87.30,10.5,0,1,66,1


In [5]:
ft1 = fantasy_team(df1)
ft1

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,PlayingXI,Points,DT,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL,Playing_xi,pickup_status
0,3692,38632,S Iyer,9428,IND,2,BAT,71.23,8.5,0,1,84,1,0,1,0,0,1,0,1,1
1,3692,38632,C Asalanka,10934,SL,5,BAT,64.60,8.5,0,1,70,1,0,1,0,0,0,1,1,1
2,3692,38632,R Sharma,576,IND,2,BAT,87.30,10.5,0,1,66,1,0,1,0,0,1,0,1,1
3,3692,38632,I Kishan,10276,IND,2,WK,73.23,8.5,0,1,141,1,0,0,0,1,1,0,1,1
4,3692,38632,V Iyer,10917,IND,2,AR,53.22,8.5,0,1,50,1,1,0,0,0,1,0,1,1
5,3692,38632,R Jadeja,587,IND,2,AR,74.73,9.0,0,1,34,1,1,0,0,0,1,0,1,1
6,3692,38632,D Shanaka,8422,SL,5,AR,50.75,9.0,0,1,32,1,1,0,0,0,0,1,1,1
7,3692,38632,B Kumar,1726,IND,2,BOWL,22.17,8.5,0,1,76,1,0,0,1,0,1,0,1,1
8,3692,38632,Y Chahal,7910,IND,2,BOWL,40.62,8.5,0,1,35,1,0,0,1,0,1,0,1,1
9,3692,38632,L Kumara,10928,SL,5,BOWL,34.56,8.5,0,1,35,1,0,0,1,0,0,1,1,1


In [6]:
sum(ft1.Credits)

97.0

Sum of points for each team in the match

In [7]:
ind_1 = ft1[ft1.Team == 'IND']

In [8]:
sl_1 = ft1[ft1.Team == 'SL']

In [9]:
s_i1 = sum(ind_1.Points)

In [10]:
s_sl1 = sum(sl_1.Points)

Total Points in the first fanatsy team

In [11]:
sum(ft1.Points)

653

# IND VS SL 2nd T20I

In [12]:
df2 = pd.read_csv('ind_sl_t20i_2nd_2022_02_26.csv')

In [13]:
df2.head()

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,DT
0,3692,38637,S Iyer,9428,IND,2,BAT,87.64,8.5,84,1,1,112,1
1,3692,38637,P Nissanka,13682,SL,5,BAT,66.77,9.5,2,0,1,100,1
2,3692,38637,R Jadeja,587,IND,2,AR,76.24,9.0,34,1,1,93,1
3,3692,38637,D Shanaka,8422,SL,5,AR,68.26,9.0,32,1,1,77,1
4,3692,38637,S Samson,8271,IND,2,WK,25.94,8.0,0,0,1,59,1


In [14]:
ft2 = fantasy_team(df2)
ft2

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,DT,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL,Playing_xi,pickup_status
0,3692,38637,S Iyer,9428,IND,2,BAT,87.64,8.5,84,1,1,112,1,0,1,0,0,1,0,1,1
1,3692,38637,P Nissanka,13682,SL,5,BAT,66.77,9.5,2,0,1,100,1,0,1,0,0,0,1,1,1
2,3692,38637,D Gunathilaka,8387,SL,5,BAT,4.45,8.5,0,0,1,56,1,0,1,0,0,0,1,1,1
3,3692,38637,S Samson,8271,IND,2,WK,25.94,8.0,0,0,1,59,1,0,0,0,1,1,0,1,1
4,3692,38637,R Jadeja,587,IND,2,AR,76.24,9.0,34,1,1,93,1,1,0,0,0,1,0,1,1
5,3692,38637,D Shanaka,8422,SL,5,AR,68.26,9.0,32,1,1,77,1,1,0,0,0,0,1,1,1
6,3692,38637,L Kumara,10928,SL,5,BOWL,46.78,8.5,35,1,1,52,1,0,0,1,0,0,1,1,1
7,3692,38637,Y Chahal,7910,IND,2,BOWL,54.21,8.5,35,1,1,39,1,0,0,1,0,1,0,1,1
8,3692,38637,B Kumar,1726,IND,2,BOWL,63.78,8.5,76,1,1,37,1,0,0,1,0,1,0,1,1
9,3692,38637,D Chameera,8393,SL,5,BOWL,72.85,9.0,30,1,1,31,1,0,0,1,0,0,1,1,1


In [15]:
sum(ft2.Credits)

96.0

Sum of points for each team in the match

In [16]:
ind_2 = ft1[ft2.Team == 'IND']

In [17]:
sl_2 = ft1[ft2.Team == 'SL']

In [18]:
s_i2 = sum(ind_2.Points)

In [19]:
s_sl2 = sum(sl_2.Points)

Total Points in the second fanatsy team

In [20]:
sum(ft2.Points)

687

# IND VS SL 3rd T20I

In [21]:
df3 = pd.read_csv('ind_sl_t20i_3rd_2022_02_27.csv')

In [22]:
df3.head()

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points
0,3692,38642,S Iyer,9428,IND,2,BAT,88.8,8.5,196.0,2,1,100
1,3692,38642,R Jadeja,587,IND,2,AR,87.6,9.0,127.0,2,1,31
2,3692,38642,S Samson,8271,IND,2,WK,85.5,8.0,59.0,1,1,35
3,3692,38642,D Shanaka,8422,SL,5,AR,82.3,9.0,109.0,2,1,105
4,3692,38642,P Nissanka,13682,SL,5,BAT,82.0,9.5,102.0,1,1,-1


In [23]:
ft3 = fantasy_team(df3)
ft3

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL,Playing_xi,pickup_status
0,3692,38642,S Iyer,9428,IND,2,BAT,88.80,8.5,196.0,2,1,100,0,1,0,0,1,0,1,1
1,3692,38642,R Sharma,576,IND,2,BAT,81.45,10.5,78.2,1,1,10,0,1,0,0,1,0,1,1
2,3692,38642,C Asalanka,10934,SL,5,BAT,57.96,8.5,76.0,1,1,9,0,1,0,0,0,1,1,1
3,3692,38642,S Samson,8271,IND,2,WK,85.50,8.0,59.0,1,1,35,0,0,0,1,1,0,1,1
4,3692,38642,D Chandimal,6245,SL,5,WK,25.80,8.5,30.0,0,1,36,0,0,0,1,0,1,1,1
5,3692,38642,D Shanaka,8422,SL,5,AR,82.30,9.0,109.0,2,1,105,1,0,0,0,0,1,1,1
6,3692,38642,C Karunaratne,7952,SL,5,AR,14.50,8.5,31.0,0,1,47,1,0,0,0,0,1,1,1
7,3692,38642,L Kumara,10928,SL,5,BOWL,60.44,8.5,87.0,2,1,60,0,0,1,0,0,1,1,1
8,3692,38642,R Bishnoi,14659,IND,2,BOWL,48.93,8.0,0.0,0,1,37,0,0,1,0,1,0,1,1
9,3692,38642,M Siraj,10808,IND,2,BOWL,42.51,8.5,0.0,0,1,41,0,0,1,0,1,0,1,1


In [24]:
sum(ft3.Credits)

94.5

Sum of points for each team in the match

In [25]:
ind_3 = ft1[ft3.Team == 'IND']

In [26]:
sl_3 = ft1[ft3.Team == 'SL']

In [27]:
s_i3 = sum(ind_3.Points)

In [28]:
s_sl3= sum(sl_3.Points)

Total Points in the third fanatsy team

In [29]:
sum(ft3.Points)

550

Average points per team (Whole series)

In [30]:
np.mean([s_i1,s_i2,s_i3]),np.mean([s_sl1,s_sl2,s_sl3])

(432.3333333333333, 220.66666666666666)

It is clear that Indian team(432.33) averages more fantasy points than Sri Lankan team(220.66)